In [2]:
from utils import authenticate

credentials, PROJECT_ID = authenticate()

print("PROJECT_ID:", PROJECT_ID)


PROJECT_ID: dotted-music-460617-k2


In [3]:
REGION = "us-central1"
print(REGION)


us-central1


In [4]:
import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    credentials=credentials
)

print("Vertex AI initialized")


C:\projects\vertex_ai_course\.venv\Lib\site-packages\google\cloud\aiplatform\models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


Vertex AI initialized


In [5]:
from google.cloud import bigquery

bq_client = bigquery.Client(
    project=PROJECT_ID,
    credentials=credentials
)

print("Bigquery Intaialized")

Bigquery Intaialized


In [6]:
QUERY_TABLES = """
SELECT table_name
FROM `bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES`
"""


query_job = bq_client.query(QUERY_TABLES)

for row in query_job:
    print(row.table_name)

posts_answers
users
posts_orphaned_tag_wiki
posts_tag_wiki
stackoverflow_posts
posts_questions
comments
posts_tag_wiki_excerpt
posts_wiki_placeholder
posts_privilege_wiki
post_history
badges
post_links
tags
votes
posts_moderator_nomination


In [6]:
INSPECT_QUERY = """
SELECT *
FROM `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 3
"""

import pandas as pd

query_job = bq_client.query(INSPECT_QUERY)

df = query_job.result().to_arrow().to_pandas()
df.head()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,16582481,Dynamic Oracle Statement,<p>Why do I always get the below Error?</p>\n\...,NaN,0,7,NaT,2013-05-16 08:27:12.230000+00:00,NaN,2018-12-10 03:16:35.247000+00:00,2018-12-10 03:16:35.247000+00:00,None,1033581.0,None,2389051,None,1,0,oracle,256
1,16587638,:hover not working to show another image,"<p>I have 2 images, initially i display the sm...",16587716.0,2,0,NaT,2013-05-16 12:36:55.087000+00:00,NaN,2013-05-16 12:43:41.440000+00:00,NaT,None,NaN,None,2168548,None,1,0,html|css|hover,256
2,16589108,MySQL Fill with sample data (simple),"<p>I'm completely new to MySQL, and was just w...",NaN,3,2,NaT,2013-05-16 13:42:14.250000+00:00,0.0,2013-05-16 14:22:50.030000+00:00,NaT,None,NaN,None,1724421,None,1,0,mysql,256


In [12]:
QUERY = """
SELECT
    CONCAT(q.title, q.body) AS input_text,
    a.body AS output_text
FROM 
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id IS NOT NULL
    AND REGEXP_CONTAINS(q.tags, "python")
    AND a.creation_date >= "2020-01-01"
LIMIT 10000
"""

query_job = bq_client.query(QUERY)
df = query_job.result().to_arrow().to_pandas()
df.head(2)

,input_text,output_text
0,categorical y-axis and datetime x-axis with Bo...,<pre><code> for py_file in glob.glob(f&quot...
1,categorical y-axis and datetime x-axis with Bo...,<p>Apache Commons is a Java library. I don't t...


In [8]:
INSTRUCTION_TEMPLATE = """Please answer the following Stackoverflow question on Python.
Answer it like you are a developer answering Stackoverflow questions.

Stackoverflow question:
"""

df["input_text_instruct"] = INSTRUCTION_TEMPLATE + df["input_text"]


In [9]:
from sklearn.model_selection import train_test_split

train, evaluation = train_test_split(
    df,
    test_size=0.2,
    random_state=42
)

In [10]:
import datetime

date = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

cols = ["input_text_instruct", "output_text"]

train.to_json(
    f"tune_data_stack_overflow_python_qa-{date}.json1",
    orient="records",
    lines=True
)

evaluation.to_json(
    f"tune_eval_data_stack_overflow_python_qa-{date}.json1",
    orient="records",
    lines=True
)